# Adding your own runoff product to the CESM (Advanced)
To add your own runoff product to the CESM, you need three things. A Stream Definition File, an ESMF Mesh file, and the dimensions of your raw data.

## ESMF Mesh File
An ESMF Mesh File can be created like below, using CrocoDash Grids.

In [ ]:
from CrocoDash.grid import Grid
from CrocoDash.topo import Topo
import xarray as xr

grid = Grid(
    lenx=360,
    leny       = 150,         # grid length in y direction
    cyclic_x=True   ,    
    ystart     = -60,       # start/end 10 degrees above/below poles to avoid singularity
    resolution = 0.10,
    name = "GLOFAS",
)
topo = Topo(grid, min_depth = 0)
topo.set_flat(10)
topo.write_esmf_mesh("<path>")

## Stream Definition File

In your case directory, copy the following file and name it `drof.streams.xml`:

```xml
<?xml version="1.0"?>
<file id="stream" version="2.0">

  <stream_info name="rof.<PRODUCT_NAME>">
    <taxmode>cycle</taxmode>
    <tintalgo>upper</tintalgo>
    <readmode>single</readmode>
    <mapalgo>bilinear</mapalgo>
    <dtlimit>3.0</dtlimit>
    <year_first>"START_YEAR"</year_first>
    <year_last>"END_YEAR"</year_last>
    <year_align>"START_YEAR"</year_align>
    <vectors>null</vectors>
    <meshfile>"PATH_TO_THE_PREVIOUSLY_GENERATED_ESMF_MESH_FILE"</meshfile>
    <lev_dimname>null</lev_dimname>
    <datafiles>
      <file>"PATH_TO_RAW_DATA_FILE_IN_NETCDF3_64BIT_OFFSET"</file>
    </datafiles>
    <datavars>
      <var>"NETCDF_VARIABLE_NAME_IN_FILE" Forr_rofl</var>
    </datavars>
    <offset>0</offset>
  </stream_info>

</file>
```

### Additional Notes

1. Update the CESM `components/cdeps/drof` directory.  
   The following entry should already exist in `namelist_definition_drof.xml`:

```xml
<value drof_mode="<PRODUCT_NAME>">rof.<PRODUCT_NAME></value>
```

2. Update `config_component` with the new `<PRODUCT_NAME>`:

```xml
<desc rof="DROF[%NULL][%NYF][%<PRODUCT_NAME>]...">
```

Make sure to replace placeholders (`<PRODUCT_NAME>`, `START_YEAR`, `END_YEAR`, file paths, and variable names) with your actual values.


## Add the dimensions
Make the following xml changes in your case directory:

```
./xmlchange ROF_NY=1500

./xmlchange ROF_NX=3600

./xmlchange ROF_DOMAIN_MESH=<MESH PATH>
```

There may be a few additional changes in the CESM, but this covers a majority of them.

Then, you can add the mapping files generated through CrocoDash to your case (which requires a passed in ESMF mesh file)